In [1]:
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from PIL import Image
from pandas import Series, DataFrame

In [2]:
data = pd.read_csv('train.csv', index_col=0)

In [3]:
data_ = np.array(data)
label=data.loc[:,"digit":"letter"].values
x_data=data.loc[:,"0":"783"].values

In [4]:
train_data_x, val_data_x, train_data_label, val_data_label = train_test_split(x_data, label, 
                                                                        test_size=0.2)

In [5]:
aug_f = transforms.Compose([transforms.Resize((28, 28)),
                            transforms.RandomAffine(30,translate=(0.1,0.1) ,
                                                    shear=10, scale=(0.7, 1.3))])

In [6]:
val_data_x=val_data_x.reshape(len(val_data_x),28,28)
val_data_x=val_data_x.astype(float)

train_data_x=train_data_x.reshape(len(train_data_x),28,28)
train_data_x=train_data_x.astype(float)

In [7]:
aug_img_val = []
aug_img_train = []
val_label=[]
train_label=[]

for i in range(len(val_data_x)):
    for _ in range(50):
        aug_img_val.append(np.array(aug_f(Image.fromarray(val_data_x[i]))))
        val_label.append(val_data_label[i])
aug_img_val=np.array(aug_img_val)
val_label=np.array(val_label)

for i in range(len(train_data_x)):
    for _ in range(50):
        aug_img_train.append(np.array(aug_f(Image.fromarray(train_data_x[i]))))
        train_label.append(train_data_label[i])
aug_img_train=np.array(aug_img_train)
train_label=np.array(train_label)

In [8]:
aug_img_val=aug_img_val.reshape(len(aug_img_val),-1)
aug_img_train=aug_img_train.reshape(len(aug_img_train),-1)

In [9]:
aug_data_val=np.hstack((val_label, aug_img_val))
aug_data_train=np.hstack((train_label, aug_img_train))

In [10]:
np.take(aug_data_val,np.random.permutation(aug_data_val.shape[0]),
        axis=0,out=aug_data_val)
np.take(aug_data_train,np.random.permutation(aug_data_train.shape[0]),
        axis=0,out=aug_data_train)

array([[4, 'F', 0.0, ..., 0.0, 0.0, 0.0],
       [6, 'Y', 4.0, ..., 1.0, 3.0, 3.0],
       [2, 'I', 0.0, ..., 1.0, 3.0, 1.0],
       ...,
       [5, 'E', 0.0, ..., 3.0, 0.0, 0.0],
       [7, 'C', 0.0, ..., 4.0, 2.0, 4.0],
       [2, 'V', 4.0, ..., 3.0, 4.0, 4.0]], dtype=object)

In [11]:
data_val=DataFrame(aug_data_val, columns=data.columns)
data_train=DataFrame(aug_data_train, columns=data.columns)

In [12]:
data_val

,digit,letter,0,1,2,3,4,5,6,7,...,774,775,776,777,778,779,780,781,782,783
0,6,L,0,0,0,4,3,1,4,2,...,1,0,0,0,0,0,0,0,0,0
1,1,L,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,P,0,0,0,0,0,0,0,0,...,3,3,3,1,1,0,4,0,0,0
3,6,U,3,1,4,2,2,8,4,38,...,4,3,0,2,0,4,4,4,0,0
4,9,E,0,0,0,4,0,3,0,0,...,0,2,1,4,4,4,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20495,0,Z,0,0,0,2,1,4,4,2,...,0,0,0,0,0,0,0,0,0,0
20496,6,O,0,0,0,0,0,0,0,0,...,0,4,0,0,4,0,4,0,0,0
20497,2,N,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20498,0,G,0,0,0,0,0,0,0,1,...,19,8,12,2,2,2,0,4,3,3


In [13]:
data_train

,digit,letter,0,1,2,3,4,5,6,7,...,774,775,776,777,778,779,780,781,782,783
0,4,F,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
1,6,Y,4,1,4,4,2,2,3,1,...,4,4,1,0,3,3,0,1,3,3
2,2,I,0,0,0,0,0,4,4,0,...,2,0,4,2,1,0,4,1,3,1
3,8,T,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,B,0,3,3,2,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81895,1,S,0,0,0,0,0,0,0,0,...,4,0,3,1,2,1,0,0,0,0
81896,1,M,0,0,0,0,0,1,2,2,...,0,3,4,0,0,0,0,0,0,0
81897,5,E,0,0,0,0,0,0,2,2,...,58,41,86,41,16,3,3,3,0,0
81898,7,C,0,1,4,4,3,3,2,0,...,6,0,3,3,2,3,4,4,2,4


In [14]:
data_val.to_csv("val_augmented.csv")
data_train.to_csv("train_augmented.csv")